![alt text](https://github.com/Laboratory-ml/Titanic/blob/master/img/2.png?raw=true)

Prowadzący **Paweł Lubiński** administrator i projektant sieci w branży produkcyjnej. Pasjonat matematyki i skrajny pragmatyk.



# Temat: Titanic - Machine Learning from Disaster - konkurs na platformie Kaggle.com

![alt text](https://github.com/dataworkshop/webinar-titanic/raw/3ff91e2d5e3c11eb11727b36d5625277bb0dd6a9/images/route.png)

![alt text](https://github.com/dataworkshop/webinar-titanic/raw/3ff91e2d5e3c11eb11727b36d5625277bb0dd6a9/images/iceberg_titanic.jpg)

![alt text](https://github.com/dataworkshop/webinar-titanic/raw/3ff91e2d5e3c11eb11727b36d5625277bb0dd6a9/images/life_boats.jpg)

![alt text](https://github.com/dataworkshop/webinar-titanic/raw/3ff91e2d5e3c11eb11727b36d5625277bb0dd6a9/images/lifeboats.png)

Na Titanicu było około 1300 pasażerów oraz 900 osób załogi. Co daje ponad 2200 ludzi.

Niestety było tam tylko 20 łodzi ratunkowych, w każdej tylko 65 miejsc.

Od początku było wiadomo, że duża część osób zginie.







In [0]:
# Klonowanie danych z repozytorium
! git clone https://github.com/Laboratory-ml/Titanic

Import potrzebnych bibliotek

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_validate

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import files

np.random.seed(2019)
%matplotlib inline

Wczytywanie danych z plików CSV

In [0]:
df_train = pd.read_csv('Titanic/input/train.csv')
df_test = pd.read_csv('Titanic/input/test.csv')

print("df_train ma rozmiar: ", df_train.shape)
print("df_test ma rozmiar: ", df_test.shape)

Zobaczmy co siedzi w danych

In [0]:
df_train.head()

In [0]:
df_test.head()


# Features / Cechy:

* PassengerId - A numerical id assigned to each 
* Survived - Czy dana osoba przeżyła. 0 - Nie, 1 - Tak. Ta wartość pojawia się tylko w test
* Pclass - Klasa socio-economic (SEC), 1- Upper, 2 - Middle, 3rd - Lower
* Name - Imię i nazwisko oddzielone przecinkiem
* Sex - Płeć, ‘male’ - męzczyzna , ‘female’- kobieta
* Age - Wiek
* SibSp - (siblink) liczba rodzeństwa (brat i siostra) / (spouses) małżonków na pokładzie
* Parch - Liczba rodziców / dzieci na pokładzie Notka: Dziecko z nianią ma tutaj przypisaną wartość 0
* Ticket - Nr biletu
* Fare - Cena biletu
* Cabin - Nr kabiny
* Embarked - Port, w którym wsiadał pasażer (C = Cherbourg, Q = Queenstown, S = Southampton)

In [0]:
df_train.info()

In [0]:
df_test.info()

Łączenie df_train i df_test w jeden obiekt df_all

In [0]:
df_all = pd.concat([df_train, df_test], sort=False)

In [0]:
df_all.info()

In [0]:
df_all.head()

In [0]:
df_all.tail()

In [0]:
!pip install missingno

In [0]:
import missingno as msno

In [0]:
msno.matrix(df_all);

In [0]:
msno.bar(df_all);

In [0]:
msno.dendrogram(df_all);

In [0]:
df_all.select_dtypes(include=[np.int, np.float]).head()

In [0]:
def get_feats(df):
    feats = df.select_dtypes(include=[np.int64]).columns.values
    black_list = ['PassengerId', 'Survived']
    
    return [feat for feat in feats if feat not in black_list]


def get_models():
    return [
        ('Logistic Regression', LogisticRegression(solver='lbfgs', max_iter=400)),
        ('Decision Tree Classifier', DecisionTreeClassifier()),
        ('Random Forest Classifier', RandomForestClassifier(n_estimators=100)),
        ('Extra Trees Classifier', ExtraTreesClassifier(n_estimators=100)),
    ]

SURV = 891
df_all[SURV:SURV+10:2]

Funkcja rysująca wykres walidacji krzyżowej

In [0]:
def plot_result(model_name, result, ylim=(0, 1.)):
    mean_train = np.round( np.mean(result['train_score']), 2 )
    mean_test = np.round( np.mean(result['test_score']), 2 )
    
    plt.title('{0}: cross validation\nmean-train-acc:{1}\nmean-test-acc:{2}'.format(model_name, mean_train, mean_test))
    plt.plot( result['train_score'], 'r-o', label="train" )
    plt.plot( result['test_score'], 'g-o', label="test" )
    plt.legend(loc='best')
    plt.ylabel('Accuracy')
    plt.xlabel('# of fold')
    plt.ylim(*ylim)
    plt.show()

# Sprawdzian krzyżowy - na cholerę mi to!?

![alt text](https://miro.medium.com/max/2736/1*rgba1BIOUys7wQcXcL4U5A.png)

Sprawdzian krzyżowy (walidacja krzyżowa, kroswalidacja, sprawdzanie krzyżowe) – metoda statystyczna, polegająca na podziale próby statystycznej na podzbiory, a następnie przeprowadzaniu wszelkich analiz na niektórych z nich (zbiór uczący), podczas gdy pozostałe służą do potwierdzenia wiarygodności jej wyników (zbiór testowy, zbiór walidacyjny).

In [0]:
def cross_val_plot(df_all):
  
  feats = get_feats(df_all)

  X = df_all[:SURV][feats].values
  y = df_all[:SURV]['Survived'].values

  for model_name, model in get_models():
    result = cross_validate(model, X, y, scoring='accuracy', cv=4, return_train_score=True)
    # print(model_name)
    plot_result(model_name, result)

In [0]:
cross_val_plot(df_all)

Predykcja - oszacujmy kto ze zbioru testowego przeżyje

In [0]:
def make_prediction(df_all, model, output_file_name):
    
    sub = df_all[SURV:].copy()
    feats = get_feats(df_all)

    X_train = df_all[:SURV][feats].values
    y_train = df_all[:SURV]['Survived'].values
    X_test = df_all[SURV:][feats]

    print(model)
    model.fit(X_train, y_train)

    sub['Survived'] = model.predict(X_test).astype(int)
    sub[ ['PassengerId', 'Survived'] ].to_csv(output_file_name, index=False)
    files.download(output_file_name)

In [0]:
 df_all[get_feats(df_all)].head()

# Wykonanie pierwszej predykcji, zapis wyników do pliku CSV

In [0]:
make_prediction(df_all, LogisticRegression(solver='lbfgs'), 'LogisticRegression.csv')

Wynik publiczny dla LogisticRegression po przesłaniu wyników: **0.66985**

In [0]:
make_prediction(df_all, DecisionTreeClassifier(), 'DecisionTreeClassifier.csv')

Wynik publiczny dla DecisionTreeClassifier po przesłaniu wyników: **0.68421**

## OK, ale jak działają te dwa modele? 

**Regresja Logistyczna**

Ogólnie rzecz ujmując, regresja logistyczna jest modelem matematycznym, pozwalającym opisać
wpływ kilku zmiennych X1, X2,...Xk na dychotomiczną zmienną Y. Model regresji logistycznej oparty
jest o funkcję logistyczną postaci:

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/9537e778e229470d85a68ee0b099c08298a1a3f6)

Funkcja logistyczna przyjmuje wartości od 0 (gdy x dąży do minus nieskończoności) do 1 (gdy x
zmierza do plus nieskończoności): 
![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/1024px-Logistic-curve.svg.png)

 Własności funkcji logistycznej są następujące:

• Przyjmuje wartości od 0 do 1. Model może opisywać wartości prawdopodobieństwa, najczęściej określającego ryzyko zachorowania lub szansę na wyzdrowienie.

• Kształt funkcji przypomina rozciągniętą literę S. Oznacza to, że do osiągnięcia pewnej
wartości progowej zmiany wartości funkcji są minimalne, potem gwałtownie wzrastają do 1
i utrzymują się na bardzo wysokim poziomie (bliskim 1).



---



**Klasyfikator oparty o drzewo decyzyjne**

Klasyfikator drzewa decyzyjnego pozwala na podejmowanie nieliniowych decyzji za pomocą prostych pytań liniowych.


![alt text](https://miro.medium.com/max/663/1*xGsYc6aXehD7lyoLEn-mMA.png)



Drzewo w takiej postaci odzwierciedla, w jaki sposób na podstawie atrybutów były podejmowane decyzje klasyfikujące (dla uproszczenia połączono niektóre gałęzie). Zaletą tej reprezentacji jest jej czytelność dla człowieka. W prosty sposób można przekształcić ją do reprezentacji regułowej.

![alt text](https://miro.medium.com/max/2592/0*1IMzGS56ZIZpY2QI)

# Feature engineering - czyli zabawa z danymi

Popatrzmy na dane, odszukajmy zależności

In [0]:
f,ax=plt.subplots(1,2,figsize=(18,8))
df_all['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived',data=df_all,ax=ax[1])
ax[1].set_title('Survived')
plt.show()

#  Sex, Survived vs Dead

In [0]:
f,ax=plt.subplots(1,2,figsize=(18,8))
df_all[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex',hue='Survived',data=df_all,ax=ax[1])
ax[1].set_title('Sex:Survived vs Dead')
plt.show()

To wygląda interesująco. Liczba mężczyzn na statku jest znacznie większa niż liczba kobiet. 

Mimo to liczba uratowanych kobiet jest prawie dwa razy większa niż uratowanych mężczyzn. 

Wskaźniki przeżycia kobiet na statku wynoszą około 75%, a mężczyzn około 18–19%.


**Dokonamy teraz modyfikacji zwanej Factorize**


In [0]:
df_all['Sex_cat'] = pd.factorize( df_all['Sex'] )[0]

In [0]:
df_all.head()

# Pclass

In [0]:
f,ax=plt.subplots(1,2,figsize=(18,8))
df_all['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'],ax=ax[0])
ax[0].set_title('Number Of Passengers By Pclass')
ax[0].set_ylabel('Count')
sns.countplot('Pclass',hue='Survived',data=df_all,ax=ax[1])
ax[1].set_title('Pclass:Survived vs Dead')
plt.show()

Wyraźnie widać, że Passenegers Of Pclass 1 otrzymali bardzo wysoki priorytet podczas ratowania. 
Mimo że liczba pasażerów w klasie 3 była znacznie wyższa, wciąż liczba przeżywających je osób jest bardzo niska, około 25%.

Dla Pclass 1% przeżył około 63%, podczas gdy dla Pclass2 około 48%. Więc pieniądze i status mają znaczenie.
Taki materialistyczny świat.

In [0]:
sns.factorplot('Pclass','Survived', hue='Sex', data=df_all)
plt.show()

# Age

In [0]:
print('Najstarszy pasażer:', df_all['Age'].max(),'lat')
print('Najmłodszy pasażer:', df_all['Age'].min(),'lat')
print('Średnia wieku', df_all['Age'].mean(),'lat')

In [0]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sns.violinplot("Pclass","Age", hue="Survived", data=df_all, split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex","Age", hue="Survived", data=df_all, split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

Obserwacje:

1) Liczba dzieci wzrasta wraz z Pclass, a wskaźnik przeżycia dla osób w wieku poniżej 10 lat (tj. dzieci) wydaje się być dobry, niezależnie od Pclass.

2) Szanse na przeżycie dla pasażerów w wieku 20-50 lat z Pclass1 są wysokie i są najlepsze dla kobiet.

3) W przypadku mężczyzn szanse na przeżycie zmniejszają się wraz ze wzrostem wieku.

Jak widzieliśmy wcześniej, cecha **Age** ma wiele wartości zerowych. Aby zastąpić te wartości NaN, możemy przypisać im średni wiek zestawu danych - ale czy todobre wyjście?

Problem polega jednak na tym, że było wielu ludzi w różnym wieku. Po prostu nie możemy przypisać 4-letniego dziecka w średnim wieku, który wynosi 29 lat. Czy jest jakiś sposób, aby dowiedzieć się, w jakim przedziale wiekowym leży pasażer?

Możemy sprawdzić cechę Name. Patrząc na tę cechę, widzimy, że imiona mają przedrostek, takie jak Pan lub Pani. W ten sposób możemy przypisać średnie wartości Panu i Pani do odpowiednich grup.



# Name

In [0]:
df_all['Initial']=0
for i in df_all:
    df_all['Initial']=df_all.Name.str.extract('([A-Za-z]+)\.')

Okej, więc tutaj używamy Regex: [A-Za-z] +) . Szukamy ciągów, które leżą między A-Z lub a-z, po których następuje ".". Dlatego z powodzeniem wyodrębniamy inicjały z nazwy.

In [0]:
pd.crosstab(df_all.Initial,df_all.Sex).T.style.background_gradient(cmap='summer_r')

Sprawdzamy występujące wartości

In [0]:
df_all['Initial'].value_counts()


Więc jest kilka błędnie napisanych przedrostków, takich jak Mlle lub Mme, które oznaczają Miss. Zastąpię je Miss i to samo dla innych wartości.

In [0]:
df_all['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'], inplace=True)

In [0]:
df_all['Initial'].value_counts()

Factorize Initial

In [0]:
df_all['Initial_cat'] = pd.factorize( df_all['Initial'] )[0]
df_all.head()

# Age

Uzupełnianie brakujących wartości

In [0]:
df_all.groupby('Initial')['Age'].mean()

Uzupełnianie brakujących wartości Age

In [0]:
df_all.loc[(df_all.Age.isnull())&(df_all.Initial=='Mr'),'Age']=33
df_all.loc[(df_all.Age.isnull())&(df_all.Initial=='Mrs'),'Age']=36
df_all.loc[(df_all.Age.isnull())&(df_all.Initial=='Master'),'Age']=5
df_all.loc[(df_all.Age.isnull())&(df_all.Initial=='Miss'),'Age']=22
df_all.loc[(df_all.Age.isnull())&(df_all.Initial=='Other'),'Age']=46

Sprawdzamy czy są brakujące dane

In [0]:
df_all.Age.isnull().any()

In [0]:
f,ax=plt.subplots(1,2,figsize=(20,10))
df_all[df_all['Survived']==0].Age.plot.hist(ax=ax[0], bins=20, edgecolor='black', color='red')
ax[0].set_title('Survived= 0')
x1=list(range(0,85,5))
ax[0].set_xticks(x1)
df_all[df_all['Survived']==1].Age.plot.hist(ax=ax[1], color='green', bins=20, edgecolor='black')
ax[1].set_title('Survived= 1')
x2=list(range(0,85,5))
ax[1].set_xticks(x2)
plt.show()

Obserwacje:

1) Małe dzieci (wiek poniżej 5 lat) zostały uratowane w dużych ilościach.Zadziałały zasady dotyczące kobiet i dzieci w pierwszej kolejności.

2) Uratowano najstarszego pasażera - 80 lat.

3) Maksymalna liczba zgonów dotyczyła osób w wieku 30–40 lat.

Teraz utwórzmy nową cechę **AgeCategory**, podzielmy ludzi ze względu na przedziały wiekowe:

In [0]:
df_all['AgeCategory'] = pd.cut(df_all['Age'],[0,9,18,30,40,50,100], labels=[9,18,30,40,50,100]) # Add column with range of Age
df_all['AgeCategory'] = df_all['AgeCategory'].cat.codes
df_all.head()

# Embarked

In [0]:
pd.crosstab([df_all.Embarked,df_all.Pclass],[df_all.Sex,df_all.Survived], margins=True).style.background_gradient(cmap='summer_r')

In [0]:
f,ax=plt.subplots(2,2,figsize=(20,15))
sns.countplot('Embarked',data=df_all,ax=ax[0,0])
ax[0,0].set_title('No. Of Passengers Boarded')
sns.countplot('Embarked',hue='Sex',data=df_all,ax=ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked')
sns.countplot('Embarked',hue='Survived',data=df_all,ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot('Embarked',hue='Pclass',data=df_all,ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2,hspace=0.5)
plt.show()

Obserwacje:

1) Maksymalna liczba osób wchodzących na pokład pochodziła z portu S. Większość z nich pochodzi z Pclass3.

2) Pasażerowie z C wyglądają na szczęściarzy, ponieważ spora ich część przeżyła. Powodem tego może być uratowanie wszystkich pasażerów Pclass1 i Pclass2.

3) Embark S wygląda na port, z którego weszła większość bogatych ludzi. Jednak szanse na przeżycie są tutaj niskie, ponieważ wielu pasażerów z Pclass3 około 81% nie przeżyło.

4) W porcie Q prawie 95% pasażerów pochodziło z Pclass3.

In [0]:
sns.factorplot('Pclass','Survived', hue='Sex', col='Embarked', data=df_all)
plt.show()


Obserwacje:

1) Szanse na przeżycie są prawie 1 dla kobiet dla Pclass1 i Pclass2, niezależnie od Pclass.

2) Port S wydaje się być bardzo pechowy dla Pclass3, ponieważ wskaźnik przeżycia zarówno dla mężczyzn jak i kobiet jest bardzo niski.

3) Wygląda na to, że port Q wydaje się być najszczęśliwszy dla mężczyzn, ponieważ prawie wszystkie pochodzą z klasy 3.

Uzupełnianie brakujących wartości.

Jak widzieliśmy, maksymalna liczba pasażerów wsiada do portu S, zastępujemy NaN wartością S.

In [0]:
df_all['Embarked'].fillna('S',inplace=True)

Sprawdzamy czy są brakujące dane

In [0]:
df_all.Embarked.isnull().any()

**Embarked - Factorize**

In [0]:
df_all['Embarked_cat'] = pd.factorize( df_all['Embarked'] )[0]

In [0]:
df_all.head()

# SibSip

Ta cecha mówi nam czy dana osoba jest sama czy z członkami rodziny.

Rodzeństwo = brat, siostra, przyrodni brat, przyrodnia siostra

Małżonek = mąż, żona

In [0]:
pd.crosstab([df_all.SibSp],df_all.Survived).style.background_gradient(cmap='summer_r')

In [0]:
f,ax=plt.subplots(1,2,figsize=(20,8))
sns.barplot('SibSp','Survived',data=df_all,ax=ax[0])
ax[0].set_title('SibSp vs Survived')
sns.factorplot('SibSp','Survived',data=df_all,ax=ax[1])
ax[1].set_title('SibSp vs Survived')
plt.close(2)
plt.show();

In [0]:
pd.crosstab(df_all.SibSp,df_all.Pclass).style.background_gradient(cmap='summer_r')


Obserwajce:

Wykres słupkowy i czynnikowy pokazują, że jeśli pasażer jest sam na pokładzie bez rodzeństwa, ma 34,5% szans na przeżycie. Wykres z grubsza maleje, jeśli liczba rodzeństwa wzrasta. To ma sens. To znaczy, jeśli mam rodzinę na pokładzie, postaram się je uratować, zamiast ratować siebie. Zaskakujące jest, że przeżycie rodzin z 5-8 członkami wynosi 0%. Przyczyną może być Pclass? Pomyśl...



# Parch

Rodzic = matka, ojciec

Dziecko = córka, syn, pasierbica, pasierb

In [0]:
pd.crosstab(df_all.Parch, df_all.Pclass).style.background_gradient(cmap='summer_r')

Tabela przestawna ponownie pokazuje, że większe rodziny były w klasie Pclass3.

In [0]:
f,ax=plt.subplots(1,2,figsize=(20,8))
sns.barplot('Parch','Survived',data=df_all,ax=ax[0])
ax[0].set_title('Parch vs Survived')
sns.factorplot('Parch','Survived',data=df_all,ax=ax[1])
ax[1].set_title('Parch vs Survived')
plt.close(2)
plt.show()

Obserwacje:

Również tutaj wyniki są dość podobne. Pasażerowie z rodzicami na pokładzie mają większą szansę na przeżycie. Zmniejsza się jednak wraz ze znacznym wzrostem liczby.

Szanse na przeżycie są dobre dla kogoś, kto ma 1-3 rodziców/dzieci na statku. Bycie samemu również okazuje się śmiertelne, a szanse na przeżycie zmniejszają się, gdy ktoś ma> 4 rodziców/dzieci na statku.

# Fare

Czyli ceny biletów

In [0]:
df_all[df_all['Fare'].isnull()]

In [0]:
val = df_all[(df_all['Pclass'] == 3) 
     & (df_all['Embarked'] == 'S') 
     & (df_all['Parch'] == 0) 
     & (df_all['Sex'] == 'male')
     & (df_all['Age'] >40.0)      
      ][['Age','Fare']];
val.groupby('Age').agg(['min','max','count','mean','median'])

Można przypuszczać że osoby w tym wieku średnio płaciły między 6 a 7 funtów. Możemy w takim razie przyjąć średnią z tych dwóch wartości. Różnica między 6 a 7 jest zbyt mała aby mieć istotny wpływ w porównaniu do całego obszaru cen (od 0 do 512, gdzie średnia to 33 a mediana to 14)

In [0]:
df_all.at[152,'Fare'] =  (7.25 + 6.2375)/2;

Sprawdzamy czy są jeszcze jakieś brakujące dane w kolumnie

In [0]:
df_all.Fare.isnull().any()

In [0]:
print('Najdroższy bilet:', df_all['Fare'].max())
print('Najtańszy bilet:', df_all['Fare'].min())
print('Średnia cena biletu:', df_all['Fare'].mean())

In [0]:
f,ax=plt.subplots(1,3,figsize=(20,8))
sns.distplot(df_all[df_all['Pclass']==1].Fare,ax=ax[0])
ax[0].set_title('Fares in Pclass 1')
sns.distplot(df_all[df_all['Pclass']==2].Fare,ax=ax[1])
ax[1].set_title('Fares in Pclass 2')
sns.distplot(df_all[df_all['Pclass']==3].Fare,ax=ax[2])
ax[2].set_title('Fares in Pclass 3')
plt.show()

# Cabin

Mamy duże braki w danych z numerami kabin.
Uzupełniamy je w prosty sposób.


In [0]:
df_all['Cabin_norm'] = df_all['Cabin'].map(lambda x: 'missing' if str(x) == 'nan' else x[0] )
df_all['Cabin_norm_cat'] = pd.factorize( df_all['Cabin_norm'] )[0]

In [0]:
df_all.head()

In [0]:
df_all.isna().sum()

In [0]:
df_all.info()

# Sprawdzamy czy nasz model daje lepsze wyniki

In [0]:
def get_feats_after_fe(df):
    feats = df.select_dtypes(include=[np.int8, np.int64, np.float64]).columns.values
    black_list = ['PassengerId', 'Survived']
    
    return [feat for feat in feats if feat not in black_list]

def cross_val_plot_after_fe(df_all):
  feats_after_fe = get_feats_after_fe(df_all)

  X = df_all[:SURV][feats_after_fe].values
  y = df_all[:SURV]['Survived'].values

  for model_name, model in get_models():
    result = cross_validate(model, X, y, scoring='accuracy', cv=4, return_train_score=True)
    # print(model_name)
    plot_result(model_name, result)


In [0]:
cross_val_plot_after_fe(df_all)

In [0]:
def make_prediction_after_fe(df_all, model, output_file_name):
    
    sub = df_all[SURV:].copy()

    #df_all = feature_engineering(df_all)
    feats = get_feats_after_fe(df_all)

    X_train = df_all[:SURV][feats].values
    y_train = df_all[:SURV]['Survived'].values
    X_test = df_all[SURV:][feats]

    print(model)
    model.fit(X_train, y_train)

    sub['Survived'] = model.predict(X_test).astype(int)
    sub[ ['PassengerId', 'Survived'] ].to_csv(output_file_name, index=False)
    files.download(output_file_name)

In [0]:
feats = get_feats_after_fe(df_all)
df_all[feats].head()

# Predykcja po uzupełnieniu i transformacji danych

In [0]:
make_prediction_after_fe(df_all, LogisticRegression(solver='lbfgs', C=0.7, max_iter=2000), 'LogisticRegression_FE.csv')

Wynik publiczny dla LogisticRegression po przesłaniu wyników: **0.77033**

In [0]:
make_prediction_after_fe(df_all, DecisionTreeClassifier(min_samples_leaf=2), 'DecisionTreeClassifier_FE.csv')

Wynik publiczny dla DecisionTreeClassifier po przesłaniu wyników: **0.75119**

In [0]:
make_prediction_after_fe(df_all, RandomForestClassifier(n_estimators=600, max_depth=8), 'RandomForestClassifier_FE.csv')

Wynik publiczny dla LogisticRegression po przesłaniu wyników: **0.78947** miejsce 3390 na 14531



# Co teraz ?


![alt text](https://miro.medium.com/max/500/1*ZhYNqU2y96_f3QkWq9oiWQ.jpeg)

# 1. Zapytaj czy możesz uzyskać więcej danych.
# 2. Zadbaj o wiedzę domenową, poznaj lepiej swój problem.
# 3. Pomyśl czy możesz lepiej przygotować swoje dane? Może coś przeoczyłeś? 
# 4. Stwórz nowe cechy, jeszcze raz zbadaj powiązania, może postaraj się o grupowanie. 
# 5. Dobierz odpowiedni model. Musisz wykonać szereg eksperymetów.
# 6. Dokonaj tuningu hiperparametrów modelu, ale tylko na etapie końcowym. Kręcenie parametrami to nie rozwiązanie problemu słabych danych.

# Na koniec mamy małą prośbę. Uzupełnij dla nas ankietę, pozwoli nam to przygotwać dla Was ciekawe spotkania. Na końcu ankiety zamieszczamy dla Was kilka ciekawych źródeł informacji. Dziękujemy!

# [**ANKIETA - kliknij aby przejść do formularza**](https://docs.google.com/forms/d/e/1FAIpQLSdGDxdiFWPtpiSKQlFSqiLEPsfjQtiwsHYCpWcxqQz3Ehd7dw/viewform)

![alt text](https://adaptastraining.com/wp-content/uploads/2016/07/thank-you-1024x752.png)